# 4 - Saccade Identification

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [21, 8]
plt.rcParams['figure.dpi'] = 100

In [30]:
from bsp.core import load_study, calibration, differentiate

study = load_study("data/prueba_fixed.bsp")

hor10 = study[1].horizontal_channel
hor10_stim = study[1].horizontal_stimuli
vel10 = medfilt(differentiate(hor10), 51)

v_mean = vel10.mean()
v_std = vel10.std()

plt.plot(abs(vel10))
# plt.axhline(v_mean - v_std, color='r')
plt.axhline(v_mean + v_std, color='r')

In [38]:
from typing import Iterator
from numpy import ndarray
from scipy.signal import medfilt
from bsp.core import impulses, differentiate

def saccades(
    channel: ndarray, 
    angle: int,
    tolerance: float = 0.2,
) -> Iterator[tuple[int, int]]:
    velocities = medfilt(differentiate(channel), 71)
    threshold = velocities.std()
    velocities = abs(velocities)
    right = len(channel) - 1

    delta_amplitude = angle * tolerance
    min_amplitude, max_amplitude = angle - delta_amplitude, angle + delta_amplitude

    idx = 0
    onset = None
    offset = None
    while idx <= right:
        if velocities[idx] > threshold:
            if onset is None:
                onset = idx
            offset = idx
        elif onset is not None:
            onset_value = channel[onset]
            offset_value = channel[offset]

            while onset > 0 and velocities[onset] > velocities[onset - 1]:
                onset -= 1

            while offset < right and velocities[offset] > velocities[offset + 1]:
                offset += 1

            window = channel[onset:offset + 1]
            amplitude = window.max() - window.min()
            
            if min_amplitude <= amplitude <= max_amplitude:
                yield onset, offset

            onset = None
            offset = None
        idx += 1

test = study[4]
hor = test.horizontal_channel
stim = test.horizontal_stimuli

plt.plot(stim, color='k', alpha=0.5)
plt.plot(hor, color='b')
for onset, offset in saccades(hor, test.angle):
    plt.axvspan(onset, offset, facecolor='r', alpha=0.2)